In [81]:
import scipy.io
import numpy as np
import collections
import pandas as pd

In [105]:
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
branch = scipy.io.loadmat('MATPOWER/branch.mat')['brach']
gen = scipy.io.loadmat('MATPOWER/gen.mat')['gen']
gencost = scipy.io.loadmat('MATPOWER/gencost.mat')['gencost']

In [106]:
power_demand = bus[:, 2][bus[:,2] != 0.0] 
num_buses = len(power_demand)
line_to_nodes = branch[:, [0, 1]] -1
num_lines = len(line_to_nodes) # I get 10823, the paper says there are 10162 lines and 661 transformers
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
for i, buses in enumerate(line_to_nodes):
    graph[buses[0]].append((buses[1], i))
    graph_reverse[buses[1]].append((buses[0], i))

In [107]:
df = pd.read_csv("GIS/CATS_gens.csv")
df = df.drop_duplicates(subset=['bus'])
gen_fuel_type = df['FuelType'].to_numpy()

In [121]:
#_, idx = np.unique(gen[:,0], return_index=True) # generator and bus are mismatch by 1 index. ie. gen 745 = bus[744]
power_generation = []
generator = []
#map(int, generator)
gen_cost = [] # simply sum up the cost
num_gen = len(generator)
seen = set()
for i, row in enumerate(gen):
    if row[0] < 2472 and row[0] not in seen:
        generator.append(row[0])
        power_generation.append(row[1])
        gen_cost.append(sum(gencost[i][1:7]))
        seen.add(row[0])
num_gen = len(generator)

In [126]:
generator = gen[:,0][gen[:,0] < 2472]
len(set(generator))

1126

In [122]:
num_gen

1126

In [119]:
max(generator)

8867.0

In [109]:
print("Number of transmission lines and transformers: " + str(num_lines))
print("Number of buses: " + str(num_buses))
print("Number of generators: " + str(num_gen))

Number of transmission lines and transformers: 10823
Number of buses: 2472
Number of generators: 2197


In [110]:
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)
for i, g in enumerate(generator):
    visited = set()
    dfs(g, g, visited)

In [118]:
line_prop_mat=np.zeros((num_gen, num_lines), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_buses), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(generator).index(list(v)[0])
        bus_prop_mat[idx][int(k)] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(generator).index(list(v)[0])
        line_prop_mat[idx][int(k)] = 1.0

1964.0
744.0
1587.0
58.0
1743.0
1735.0
339.0
2673.0


IndexError: index 2673 is out of bounds for axis 0 with size 2472